In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

2025-10-29 04:00:06.468298: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-29 04:00:06.485649: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-29 04:00:06.561953: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-29 04:00:06.622607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761690606.680616    1450 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761690606.69

In [2]:
import re


def process_packets(log_text: str, total_duration: float = 60.0):
    """
    Parse packet log text, extract key fields, and assign synthetic timing information.

    Args:
        log_text (str): Raw packet dump text.
        total_duration (float): Total capture duration in seconds.

    Returns:
        pandas.DataFrame: Table of packets with time-relative and time-delta columns.
    """
    # Split log into packets
    packets_raw = re.split(r'---- PACKET ----', log_text)
    packets = []

    for pkt in packets_raw:
        if not pkt.strip():
            continue

        # Extract fields
        rssi = re.search(r'RSSI:\s*(-?\d+)', pkt)
        channel = re.search(r'Channel:\s*(\d+)', pkt)
        length = re.search(r'Len:\s*(\d+)', pkt)
        da = re.search(r'DA:\s*([\dA-F:]+)', pkt, re.I)
        sa = re.search(r'SA:\s*([\dA-F:]+)', pkt, re.I)
        bssid = re.search(r'BSSID:\s*([\dA-F:]+)', pkt, re.I)

        packets.append({
            "radiotap.dbm_antsignal": int(rssi.group(1)) if rssi else None,
            "radiotap.channel.freq": int(channel.group(1)) if channel else None,
            "frame.len": int(length.group(1)) if length else None,
            "frame.cap_len": int(length.group(1)) if length else None,
            "wlan.da": da.group(1) if da else None,
            "wlan.sa": sa.group(1) if sa else None,
            "wlan.bssid": bssid.group(1) if bssid else None,
        })

    # Convert to DataFrame
    df = pd.DataFrame(packets)
    n = len(df)

    # Generate time features
    if n > 1:
        df["frame.time_relative"] = np.linspace(0, total_duration, n)
        df["frame.time_delta"] = df["frame.time_relative"].diff().fillna(0)
    else:
        df["frame.time_relative"] = 0
        df["frame.time_delta"] = 0

    return df


In [3]:
packet_data = """
---- PACKET ----
RSSI: -81 dBm  Channel: 84  Len: 60
DA: 0F:00:00:00:00:00  SA: 00:00:C7:59:00:00  BSSID: B9:29:99:A5:01:00
Payload (first 36 bytes):
0000: 00 00 04 00 10 00 0F 00 D7 00 BE 00 02 12 C6 50 
0010: 00 00 00 00 00 00 01 00 80 00 00 00 FF FF FF FF 
0020: FF FF E8 26 
-----------------

---- PACKET ----
RSSI: -83 dBm  Channel: 112  Len: 128
DA: 76:82:D0:BE:E6:F0  SA: 4C:F6:03:00:00:00  BSSID: 64:00:31:04:00:0B
-----------------

---- PACKET ----
RSSI: -89 dBm  Channel: 192  Len: 60
DA: FF:FF:FF:FF:FF:FF  SA: FF:FF:77:21:17:A9  BSSID: 77:E5:20:62:01:00
Payload (first 36 bytes):
0000: FF FF 05 00 B0 44 FF FF 44 02 34 02 0E 09 20 50 
0010: 00 00 00 00 00 00 01 00 94 00 00 00 C0 35 32 DE 
0020: FF FF E8 26 
-----------------

---- PACKET ----
RSSI: -82 dBm  Channel: 84  Len: 60
DA: 03:00:00:00:00:00  SA: 00:00:ED:B9:3A:E7  BSSID: 79:1E:F2:36:01:00
-----------------
"""


In [4]:
test=process_packets(packet_data,total_duration=4.0)
print(test)

   radiotap.dbm_antsignal  radiotap.channel.freq  frame.len  frame.cap_len  \
0                     -81                     84         60             60   
1                     -83                    112        128            128   
2                     -89                    192         60             60   
3                     -82                     84         60             60   

             wlan.da            wlan.sa         wlan.bssid  \
0  0F:00:00:00:00:00  00:00:C7:59:00:00  B9:29:99:A5:01:00   
1  76:82:D0:BE:E6:F0  4C:F6:03:00:00:00  64:00:31:04:00:0B   
2  FF:FF:FF:FF:FF:FF  FF:FF:77:21:17:A9  77:E5:20:62:01:00   
3  03:00:00:00:00:00  00:00:ED:B9:3A:E7  79:1E:F2:36:01:00   

   frame.time_relative  frame.time_delta  
0             0.000000          0.000000  
1             1.333333          1.333333  
2             2.666667          1.333333  
3             4.000000          1.333333  


In [5]:
test['frame.time_delta'] = test['frame.time_delta'].astype(int)
test['frame.time_relative'] = test['frame.time_relative'].astype(int)

In [6]:
test.dtypes

radiotap.dbm_antsignal     int64
radiotap.channel.freq      int64
frame.len                  int64
frame.cap_len              int64
wlan.da                   object
wlan.sa                   object
wlan.bssid                object
frame.time_relative        int64
frame.time_delta           int64
dtype: object

In [7]:
awid_path="AWID-CLS-R-Trn/1"

In [8]:
features = [
    'frame.interface_id',
    'frame.dlt',
    'frame.offset_shift',
    'frame.time_epoch',
    'frame.time_delta',
    'frame.time_delta_displayed',
    'frame.time_relative',
    'frame.len',
    'frame.cap_len',
    'frame.marked',
    'frame.ignored',
    'radiotap.version',
    'radiotap.pad',
    'radiotap.length',
    'radiotap.present.tsft',
    'radiotap.present.flags',
    'radiotap.present.rate',
    'radiotap.present.channel',
    'radiotap.present.fhss',
    'radiotap.present.dbm_antsignal',
    'radiotap.present.dbm_antnoise',
    'radiotap.present.lock_quality',
    'radiotap.present.tx_attenuation',
    'radiotap.present.db_tx_attenuation',
    'radiotap.present.dbm_tx_power',
    'radiotap.present.antenna',
    'radiotap.present.db_antsignal',
    'radiotap.present.db_antnoise',
    'radiotap.present.rxflags',
    'radiotap.present.xchannel',
    'radiotap.present.mcs',
    'radiotap.present.ampdu',
    'radiotap.present.vht',
    'radiotap.present.reserved',
    'radiotap.present.rtap_ns',
    'radiotap.present.vendor_ns',
    'radiotap.present.ext',
    'radiotap.mactime',
    'radiotap.flags.cfp',
    'radiotap.flags.preamble',
    'radiotap.flags.wep',
    'radiotap.flags.frag',
    'radiotap.flags.fcs',
    'radiotap.flags.datapad',
    'radiotap.flags.badfcs',
    'radiotap.flags.shortgi',
    'radiotap.datarate',
    'radiotap.channel.freq',
    'radiotap.channel.type.turbo',
    'radiotap.channel.type.cck',
    'radiotap.channel.type.ofdm',
    'radiotap.channel.type.2ghz',
    'radiotap.channel.type.5ghz',
    'radiotap.channel.type.passive',
    'radiotap.channel.type.dynamic',
    'radiotap.channel.type.gfsk',
    'radiotap.channel.type.gsm',
    'radiotap.channel.type.sturbo',
    'radiotap.channel.type.half',
    'radiotap.channel.type.quarter',
    'radiotap.dbm_antsignal',
    'radiotap.antenna',
    'radiotap.rxflags.badplcp',
    'wlan.fc.type_subtype',
    'wlan.fc.version',
    'wlan.fc.type',
    'wlan.fc.subtype',
    'wlan.fc.ds',
    'wlan.fc.frag',
    'wlan.fc.retry',
    'wlan.fc.pwrmgt',
    'wlan.fc.moredata',
    'wlan.fc.protected',
    'wlan.fc.order',
    'wlan.duration',
    'wlan.ra',
    'wlan.da',
    'wlan.ta',
    'wlan.sa',
    'wlan.bssid',
    'wlan.frag',
    'wlan.seq',
    'wlan.bar.type',
    'wlan.ba.control.ackpolicy',
    'wlan.ba.control.multitid',
    'wlan.ba.control.cbitmap',
    'wlan.bar.compressed.tidinfo',
    'wlan.ba.bm',
    'wlan.fcs_good',
    'wlan_mgt.fixed.capabilities.ess',
    'wlan_mgt.fixed.capabilities.ibss',
    'wlan_mgt.fixed.capabilities.cfpoll.ap',
    'wlan_mgt.fixed.capabilities.privacy',
    'wlan_mgt.fixed.capabilities.preamble',
    'wlan_mgt.fixed.capabilities.pbcc',
    'wlan_mgt.fixed.capabilities.agility',
    'wlan_mgt.fixed.capabilities.spec_man',
    'wlan_mgt.fixed.capabilities.short_slot_time',
    'wlan_mgt.fixed.capabilities.apsd',
    'wlan_mgt.fixed.capabilities.radio_measurement',
    'wlan_mgt.fixed.capabilities.dsss_ofdm',
    'wlan_mgt.fixed.capabilities.del_blk_ack',
    'wlan_mgt.fixed.capabilities.imm_blk_ack',
    'wlan_mgt.fixed.listen_ival',
    'wlan_mgt.fixed.current_ap',
    'wlan_mgt.fixed.status_code',
    'wlan_mgt.fixed.timestamp',
    'wlan_mgt.fixed.beacon',
    'wlan_mgt.fixed.aid',
    'wlan_mgt.fixed.reason_code',
    'wlan_mgt.fixed.auth.alg',
    'wlan_mgt.fixed.auth_seq',
    'wlan_mgt.fixed.category_code',
    'wlan_mgt.fixed.htact',
    'wlan_mgt.fixed.chanwidth',
    'wlan_mgt.fixed.fragment',
    'wlan_mgt.fixed.sequence',
    'wlan_mgt.tagged.all',
    'wlan_mgt.ssid',
    'wlan_mgt.ds.current_channel',
    'wlan_mgt.tim.dtim_count',
    'wlan_mgt.tim.dtim_period',
    'wlan_mgt.tim.bmapctl.multicast',
    'wlan_mgt.tim.bmapctl.offset',
    'wlan_mgt.country_info.environment',
    'wlan_mgt.rsn.version',
    'wlan_mgt.rsn.gcs.type',
    'wlan_mgt.rsn.pcs.count',
    'wlan_mgt.rsn.akms.count',
    'wlan_mgt.rsn.akms.type',
    'wlan_mgt.rsn.capabilities.preauth',
    'wlan_mgt.rsn.capabilities.no_pairwise',
    'wlan_mgt.rsn.capabilities.ptksa_replay_counter',
    'wlan_mgt.rsn.capabilities.gtksa_replay_counter',
    'wlan_mgt.rsn.capabilities.mfpr',
    'wlan_mgt.rsn.capabilities.mfpc',
    'wlan_mgt.rsn.capabilities.peerkey',
    'wlan_mgt.tcprep.trsmt_pow',
    'wlan_mgt.tcprep.link_mrg',
    'wlan.wep.iv',
    'wlan.wep.key',
    'wlan.wep.icv',
    'wlan.tkip.extiv',
    'wlan.ccmp.extiv',
    'wlan.qos.tid',
    'wlan.qos.priority',
    'wlan.qos.eosp',
    'wlan.qos.ack',
    'wlan.qos.amsdupresent',
    'wlan.qos.buf_state_indicated',
    'wlan.qos.bit4',
    'wlan.qos.txop_dur_req',
    'data.len',
    'class'
]


In [9]:
awid=pd.read_csv(awid_path,header=None,names=features,low_memory=False)

In [10]:
present_features = [
    'frame.len',
    'frame.cap_len',
    'radiotap.dbm_antsignal',
    'radiotap.channel.freq',
    'wlan.da',
    'wlan.sa',
    'wlan.bssid'
    'frame.time_delta', 'frame.time_relative', 'radiotap.present.channel','wlan_mgt.ds.current_channel','class' 
]


In [11]:
df = awid[[
    'frame.len',
    'frame.cap_len',
    'radiotap.dbm_antsignal',
    'radiotap.channel.freq',
    'wlan.da',
    'wlan.sa',
    'wlan.bssid',
    'frame.time_delta', 'frame.time_relative', 'radiotap.present.channel','wlan_mgt.ds.current_channel','class' 
]]

In [12]:
df.shape

(1795575, 12)

In [13]:
df.head()

,frame.len,frame.cap_len,radiotap.dbm_antsignal,radiotap.channel.freq,wlan.da,wlan.sa,wlan.bssid,frame.time_delta,frame.time_relative,radiotap.present.channel,wlan_mgt.ds.current_channel,class
0,261,0,1,0,b0:48:7a:e2:62:23,b0:48:7a:e2:62:23,0,0.000000,261,0,0,normal
0,185,0,1,0,38:22:9d:ae:c5:65,38:22:9d:ae:c5:65,0,0.024271,185,0,0,normal
0,185,0,1,0,38:22:9d:c4:8d:28,38:22:9d:c4:8d:28,0,0.001631,185,0,0,normal
0,159,0,1,0,00:13:33:87:62:6d,00:13:33:87:62:6d,0,0.055325,159,0,0,normal
0,54,0,1,0,00:25:bc:ed:07:cf,28:c6:8e:86:d3:d6,0,0.000415,54,0,?,normal


In [14]:
df.isna().sum()

frame.len                      0
frame.cap_len                  0
radiotap.dbm_antsignal         0
radiotap.channel.freq          0
wlan.da                        0
wlan.sa                        0
wlan.bssid                     0
frame.time_delta               0
frame.time_relative            0
radiotap.present.channel       0
wlan_mgt.ds.current_channel    0
class                          0
dtype: int64

In [15]:
df.replace({"?": None}, inplace=True)

/tmp/ipykernel_1450/2662744428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace({"?": None}, inplace=True)


In [16]:
df.isna().sum()

frame.len                            0
frame.cap_len                        0
radiotap.dbm_antsignal             780
radiotap.channel.freq              780
wlan.da                         452821
wlan.sa                         454197
wlan.bssid                      455389
frame.time_delta                     0
frame.time_relative                  0
radiotap.present.channel             0
wlan_mgt.ds.current_channel    1665680
class                                0
dtype: int64

In [17]:
df.describe()

,frame.len,frame.cap_len,frame.time_delta,frame.time_relative,radiotap.present.channel
count,1.795575e+06,1795575.0,1.795575e+06,1.795575e+06,1795575.0
mean,5.153149e+02,0.0,2.004690e-03,5.153149e+02,0.0
std,6.649245e+02,0.0,6.379536e-03,6.649245e+02,0.0
min,4.000000e+01,0.0,0.000000e+00,4.000000e+01,0.0
25%,4.000000e+01,0.0,3.150000e-04,4.000000e+01,0.0
50%,1.120000e+02,0.0,5.490000e-04,1.120000e+02,0.0
75%,1.552000e+03,0.0,1.113000e-03,1.552000e+03,0.0
max,1.570000e+03,0.0,3.502630e-01,1.570000e+03,0.0


In [18]:
df.head()

,frame.len,frame.cap_len,radiotap.dbm_antsignal,radiotap.channel.freq,wlan.da,wlan.sa,wlan.bssid,frame.time_delta,frame.time_relative,radiotap.present.channel,wlan_mgt.ds.current_channel,class
0,261,0,1,0,b0:48:7a:e2:62:23,b0:48:7a:e2:62:23,0,0.000000,261,0,0,normal
0,185,0,1,0,38:22:9d:ae:c5:65,38:22:9d:ae:c5:65,0,0.024271,185,0,0,normal
0,185,0,1,0,38:22:9d:c4:8d:28,38:22:9d:c4:8d:28,0,0.001631,185,0,0,normal
0,159,0,1,0,00:13:33:87:62:6d,00:13:33:87:62:6d,0,0.055325,159,0,0,normal
0,54,0,1,0,00:25:bc:ed:07:cf,28:c6:8e:86:d3:d6,0,0.000415,54,0,None,normal


In [19]:
df.shape

(1795575, 12)

In [20]:
df.describe()

,frame.len,frame.cap_len,frame.time_delta,frame.time_relative,radiotap.present.channel
count,1.795575e+06,1795575.0,1.795575e+06,1.795575e+06,1795575.0
mean,5.153149e+02,0.0,2.004690e-03,5.153149e+02,0.0
std,6.649245e+02,0.0,6.379536e-03,6.649245e+02,0.0
min,4.000000e+01,0.0,0.000000e+00,4.000000e+01,0.0
25%,4.000000e+01,0.0,3.150000e-04,4.000000e+01,0.0
50%,1.120000e+02,0.0,5.490000e-04,1.120000e+02,0.0
75%,1.552000e+03,0.0,1.113000e-03,1.552000e+03,0.0
max,1.570000e+03,0.0,3.502630e-01,1.570000e+03,0.0


In [21]:
df['class'].value_counts(normalize=True)

class
normal           0.909564
injection        0.036411
impersonation    0.027023
flooding         0.027002
Name: proportion, dtype: float64

In [22]:
df.isna().sum()

frame.len                            0
frame.cap_len                        0
radiotap.dbm_antsignal             780
radiotap.channel.freq              780
wlan.da                         452821
wlan.sa                         454197
wlan.bssid                      455389
frame.time_delta                     0
frame.time_relative                  0
radiotap.present.channel             0
wlan_mgt.ds.current_channel    1665680
class                                0
dtype: int64

In [23]:
for col in df.columns:
 df[col] = pd.to_numeric(df[col], errors='ignore')
# that makes more sense
df.select_dtypes(['number']).shape

/tmp/ipykernel_1450/2422610646.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_1450/2422610646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='ignore')


(1795575, 9)

In [24]:
df.describe()

,frame.len,frame.cap_len,radiotap.dbm_antsignal,radiotap.channel.freq,wlan.bssid,frame.time_delta,frame.time_relative,radiotap.present.channel,wlan_mgt.ds.current_channel
count,1.795575e+06,1795575.0,1794795.0,1794795.0,1.340186e+06,1.795575e+06,1.795575e+06,1795575.0,129895.000000
mean,5.153149e+02,0.0,1.0,0.0,2.164625e-03,2.004690e-03,5.153149e+02,0.0,0.161592
std,6.649245e+02,0.0,0.0,0.0,1.176390e-01,6.379536e-03,6.649245e+02,0.0,2.061830
min,4.000000e+01,0.0,1.0,0.0,0.000000e+00,0.000000e+00,4.000000e+01,0.0,0.000000
25%,4.000000e+01,0.0,1.0,0.0,0.000000e+00,3.150000e-04,4.000000e+01,0.0,0.000000
50%,1.120000e+02,0.0,1.0,0.0,0.000000e+00,5.490000e-04,1.120000e+02,0.0,0.000000
75%,1.552000e+03,0.0,1.0,0.0,0.000000e+00,1.113000e-03,1.552000e+03,0.0,0.000000
max,1.570000e+03,0.0,1.0,0.0,1.200000e+01,3.502630e-01,1.570000e+03,0.0,219.000000


In [25]:
df.dropna(inplace=True)



/tmp/ipykernel_1450/1422412847.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [26]:
for col in df.columns:
 df[col] = pd.to_numeric(df[col], errors='ignore')
# that makes more sense
df.select_dtypes(['number']).shape


/tmp/ipykernel_1450/3908241133.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_1450/3908241133.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='ignore')


(129895, 9)

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
train_df, test_df = train_test_split(
    df,
    test_size = 0.20,
    random_state = 42,
    stratify = df['class']  # replace ‘class’ with the correct target column name
)



In [30]:
x,y = df.select_dtypes(['number']), df['class']
df1=  pd.concat([x,y], axis=1)


train_df, test_df = train_test_split(
    df1,
    test_size = 0.20,
    random_state = 42,
    stratify = df['class']  # replace ‘class’ with the correct target column name
)





In [31]:
train_df.dtypes

frame.len                        int64
frame.cap_len                    int64
radiotap.dbm_antsignal         float64
radiotap.channel.freq          float64
wlan.bssid                     float64
frame.time_delta               float64
frame.time_relative              int64
radiotap.present.channel         int64
wlan_mgt.ds.current_channel    float64
class                           object
dtype: object

In [32]:
from sklearn.preprocessing import Normalizer

In [33]:
transformer = Normalizer()

In [34]:
transform = transformer.fit(train_df.drop(columns=['class']))

In [35]:
trainAfterNorm = transform.transform(train_df.drop(columns=["class"]))

In [36]:
testAferNorm = transform.transform(test_df.drop(columns=["class"]))

In [37]:
X_train,y_train = trainAfterNorm, train_df['class']

X_test,y_test= testAferNorm, test_df['class']

In [181]:
test.dtypes

radiotap.dbm_antsignal     int64
radiotap.channel.freq      int64
frame.len                  int64
frame.cap_len              int64
wlan.da                   object
wlan.sa                   object
wlan.bssid                object
frame.time_relative        int64
frame.time_delta           int64
dtype: object

In [293]:
X_train.shape

(103916, 9)

In [39]:
# Effectuez un ajustement basique avec Naive Bayes
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

# Entraînez notre modèle avec les données
nb.fit(X_train,y_train)

# Le modèle a été entraîné avec les données

,priors,None
,var_smoothing,1e-09


In [40]:
GaussianNB(priors=None, var_smoothing=1e-09)

,priors,None
,var_smoothing,1e-09


In [41]:


def get_test_accuracy_of(model):
    y_preds = model.predict(X_test)
    return accuracy_score(y_preds, y_test)


get_test_accuracy_of(nb)



0.9886831671734863

In [44]:
get_test_accuracy_of(lr)

0.9859501905385119

In [43]:


from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_test, y_test)
# Logistic Regressions does even worse
get_test_accuracy_of(lr)



0.9859501905385119

In [45]:
prediction = nb.predict(X_test)

In [46]:
normal = 0
notNormal = 0

for i in prediction:
    if i == "normal":
        normal += 1
    else:
        notNormal +=1

In [47]:
print(normal,notNormal)

25908 71


In [48]:
for i in range(0, prediction.shape[0]):
    print(f'Packet: {i} -> Prediction : {prediction[i]}')

Packet: 0 -> Prediction : normal
Packet: 1 -> Prediction : normal
Packet: 2 -> Prediction : normal
Packet: 3 -> Prediction : normal
Packet: 4 -> Prediction : normal
Packet: 5 -> Prediction : normal
Packet: 6 -> Prediction : normal
Packet: 7 -> Prediction : normal
Packet: 8 -> Prediction : normal
Packet: 9 -> Prediction : normal
Packet: 10 -> Prediction : normal
Packet: 11 -> Prediction : normal
Packet: 12 -> Prediction : normal
Packet: 13 -> Prediction : normal
Packet: 14 -> Prediction : normal
Packet: 15 -> Prediction : normal
Packet: 16 -> Prediction : normal
Packet: 17 -> Prediction : normal
Packet: 18 -> Prediction : normal
Packet: 19 -> Prediction : normal
Packet: 20 -> Prediction : normal
Packet: 21 -> Prediction : normal
Packet: 22 -> Prediction : normal
Packet: 23 -> Prediction : normal
Packet: 24 -> Prediction : normal
Packet: 25 -> Prediction : normal
Packet: 26 -> Prediction : normal
Packet: 27 -> Prediction : normal
Packet: 28 -> Prediction : normal
Packet: 29 -> Prediction

In [49]:
for i in range(0, prediction.shape[0]):
    if(prediction[i] != "normal"):
        print(f'Packet: {i} -> Prediction : {prediction[i]}')

Packet: 54 -> Prediction : flooding
Packet: 367 -> Prediction : flooding
Packet: 915 -> Prediction : flooding
Packet: 1131 -> Prediction : flooding
Packet: 1297 -> Prediction : flooding
Packet: 1527 -> Prediction : flooding
Packet: 1905 -> Prediction : flooding
Packet: 2531 -> Prediction : flooding
Packet: 3675 -> Prediction : flooding
Packet: 3755 -> Prediction : flooding
Packet: 5134 -> Prediction : flooding
Packet: 5691 -> Prediction : flooding
Packet: 5714 -> Prediction : flooding
Packet: 5857 -> Prediction : flooding
Packet: 5982 -> Prediction : flooding
Packet: 6271 -> Prediction : flooding
Packet: 7064 -> Prediction : flooding
Packet: 9035 -> Prediction : flooding
Packet: 9397 -> Prediction : flooding
Packet: 9986 -> Prediction : flooding
Packet: 10093 -> Prediction : flooding
Packet: 10364 -> Prediction : flooding
Packet: 10536 -> Prediction : flooding
Packet: 10553 -> Prediction : flooding
Packet: 11118 -> Prediction : flooding
Packet: 11785 -> Prediction : flooding
Packet: 11

In [ ]:

# # X_test = test.select_dtypes(['number'])  
# # print(X_test.columns.tolist())

# # Predict each packet in a loop
# for i in range(0, X_test.shape[0]):
#     # Reshape to 2D (since sklearn expects 2D input)
#     prediction = nb.predict(X_test[i])
#     print(f"Packet {i} → Predicted Class: {prediction}")


ValueError: Expected 2D array, got 1D array instead:
array=[7.07090631e-01 0.00000000e+00 4.77763940e-03 0.00000000e+00
 0.00000000e+00 1.66529399e-04 7.07090631e-01 0.00000000e+00
 4.77763940e-03].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.